In [8]:
%matplotlib inline
import os
import sys
sys.path.append("..")
import time
import numpy as np

from joblib import Parallel, delayed
from skimage.io import imread

from canon.pattern import LatentExtractor
from canon.util import split_workload

In [9]:
def process_img(img_files, model_name):
    extractor = LatentExtractor(model_name)
    img_data = np.array([imread(f) for f in img_files])
    img_idx = np.array([[int(f[-9:-4])] for f in img_files])
    codes = extractor.features(img_data)
    return np.hstack([img_idx, codes])

def extract_features(jpg_dir, model_name):
    dir_path = os.path.join("img", jpg_dir)
    filenames = [os.path.join(dir_path, filename) for filename in os.listdir(dir_path)]
    print('Found %d files in the directory %s.' % (len(filenames), dir_path))

    fn_grps = split_workload(filenames, 32)

    with Parallel(n_jobs=-1, verbose=12) as parallel:
        data = parallel(delayed(process_img)(grp, model_name) for grp in fn_grps)
        data = np.vstack(data)
        data = data[data[:,0].argsort()]
        print("Loaded a data of shape {}".format(data.shape))
    
    return data

In [13]:
jpg_dir = "C_5_1_2_scan1"
model_name = "ae_128_256_conv_4"

data = extract_features(jpg_dir, model_name)
np.save("features/" + jpg_dir + "_256.npy", data)

Found 900 files in the directory img/C_5_1_2_scan1.


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:    4.9s
[Parallel(n_jobs=-1)]: Done  15 tasks      | elapsed:    4.9s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    5.0s
[Paralle

Loaded a data of shape (900, 257)
